# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Blackmans Bay,-43.0167,147.3167,8.62,85,100,2.57,AU,1691678593
1,1,Grytviken,-54.2811,-36.5092,-3.76,90,93,3.76,GS,1691678593
2,2,Tiksi,71.6872,128.8694,16.83,48,7,3.29,RU,1691678593
3,3,Novodvinsk,64.4165,40.8122,26.69,51,0,1.75,RU,1691678593
4,4,Sao Joao Da Barra,-21.6403,-41.0511,25.05,75,49,6.05,BR,1691678593


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# ssYOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    alpha = 0.5,
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    scale = 0.5
)

# Display the map
# YOUR CODE HERE
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

selected_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)                              
                              ]

selected_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Novodvinsk,64.4165,40.8122,26.69,51,0,1.75,RU,1691678593
47,47,Bethel,41.3712,-73.4140,26.94,70,0,2.06,US,1691678595
50,50,Bredasdorp,-34.5322,20.0403,21.79,58,0,0.60,ZA,1691678596
111,111,Augusta,48.3667,10.8833,24.62,43,0,1.54,DE,1691678599
121,121,Dalhart,36.0595,-102.5133,23.60,68,0,4.12,US,1691678599


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[["City", "Lat", "Lng", "Humidity", "Country"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df.insert(5,"Hotel Name","")

# Display sample data
# YOUR CODE HERE
hotel_df


Index(['City', 'Lat', 'Lng', 'Humidity', 'Country', 'Hotel Name'], dtype='object')

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    # YOUR CODE HERE
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"] = f"accommodation.hotel"
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    weather_data = name_address.json()
        
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = weather_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
    

Starting hotel search
Blackmans Bay - nearest hotel: Villa Howden
Grytviken - nearest hotel: No hotel found
Tiksi - nearest hotel: Арктика
Novodvinsk - nearest hotel: Уют
Sao Joao Da Barra - nearest hotel: Pousada Mediterrâneo
Iqaluit - nearest hotel: Frobisher Inn
Port-Aux-Francais - nearest hotel: Keravel
Invercargill - nearest hotel: The Grand
Bilibino - nearest hotel: No hotel found
Remire-Montjoly - nearest hotel: Complexe Belova
Tongliao - nearest hotel: No hotel found
Vilyuchinsk - nearest hotel: Победа
College - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
Vadso - nearest hotel: Scandic Vadsø
Tchaourou - nearest hotel: Auberge
West Island - nearest hotel: Cocos Village Bungalows
Georgetown - nearest hotel: Page 63 hostel
Ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
Jamestown - nearest hotel: DoubleTree Jamestown
Steinbach - nearest hotel: Alt Steinbach
Puerto Natales - nearest hotel: Dorotea Patagonia Hostel
Itanhem - nearest hotel: No hotel found
Chernomor

Posto Fiscal Rolim De Moura - nearest hotel: No hotel found
Inarajan Village - nearest hotel: No hotel found
Stanley - nearest hotel: Hotel 52
Betio Village - nearest hotel: Seaman Hostel
Byron Bay - nearest hotel: Lord Byron Resort
Mogocha - nearest hotel: 7Ka
Cestos City - nearest hotel: No hotel found
Torghay - nearest hotel: No hotel found
Laminusa - nearest hotel: No hotel found
Longyearbyen - nearest hotel: The Vault
Saint-Pierre - nearest hotel: Tropic Hotel
Wuwei - nearest hotel: Hepingli Boutique Hotel
Usinsk - nearest hotel: общежитие Лукойл-Коми
Bonthe - nearest hotel: No hotel found
Plouzane - nearest hotel: Les Voyageurs
Olyokminsk - nearest hotel: Колос
Malango - nearest hotel: Madarana
Las Palmas - nearest hotel: Era de La Corte
Jiaojiang - nearest hotel: 台州远洲凤凰山庄
Pisco - nearest hotel: La Portada
Coquimbo - nearest hotel: Hotel Iberia
Salinas - nearest hotel: The Traveler's Hotel
Colares - nearest hotel: Hotel Arribas
Hilo - nearest hotel: Dolphin Bay Hotel
Ust-Nera - n

Amargosa - nearest hotel: No hotel found
Rongelap - nearest hotel: No hotel found
Kapa'A - nearest hotel: Pono Kai Resort
Vila Do Maio - nearest hotel: Residencial Jardins De Maio
Seymour - nearest hotel: Quality Inn & Suites
Emporia - nearest hotel: Budget Host Inn Emporia
Cheongsong Gun - nearest hotel: 주왕산온천관광호텔
Jilib - nearest hotel: No hotel found
Zakamensk - nearest hotel: Сона
Coracora - nearest hotel: Hostal San Marcos
Kirakira - nearest hotel: No hotel found
Villa Gesell - nearest hotel: Hotel El Faro
Shimoda - nearest hotel: ホテルウラガ
Namibe - nearest hotel: Hotel Chik Chik Namibe
Ormara - nearest hotel: No hotel found
Yelizovo - nearest hotel: Николина Усадьба
Egersund - nearest hotel: Grand hotell Egersund
Bandarbeyla - nearest hotel: No hotel found
Ta`U - nearest hotel: No hotel found
Akureyri - nearest hotel: Hótel Norðurland
Aginskoye - nearest hotel: No hotel found
Cidreira - nearest hotel: Hotel Castelo
Dalbandin - nearest hotel: SF Al-dawood Restaurant
Dwarka - nearest h

San Sebastian De La Gomera - nearest hotel: Hotel Villa Gomera
Olutanga - nearest hotel: No hotel found
Bovalino - nearest hotel: No hotel found
Kolasib - nearest hotel: Mizo Hotel
Brookings - nearest hotel: Quality Inn
Barcelos - nearest hotel: Hotel Dom Nuno
Yarmouth - nearest hotel: The Inn At Cape Cod
Pemangkat - nearest hotel: Hotel Kelapa Gading
Inari - nearest hotel: Villa Lanca
Maceio - nearest hotel: Maceió Hostel e Pousada
Naze - nearest hotel: ビジネス旅館畠山
Boa Esperanca Do Sul - nearest hotel: No hotel found
Severo-Yeniseyskiy - nearest hotel: Актолик
Quang Ngai - nearest hotel: Ngoc Anh
Peterhead - nearest hotel: Albert Hotel
Tikapur - nearest hotel: Banana Agro Resort


,City,Lat,Lng,Humidity,Country,Hotel Name
0,Blackmans Bay,-43.0167,147.3167,85,AU,Villa Howden
1,Grytviken,-54.2811,-36.5092,90,GS,No hotel found
2,Tiksi,71.6872,128.8694,48,RU,Арктика
3,Novodvinsk,64.4165,40.8122,51,RU,Уют
4,Sao Joao Da Barra,-21.6403,-41.0511,75,BR,Pousada Mediterrâneo
...,...,...,...,...,...,...
587,Boa Esperanca Do Sul,-21.9925,-48.3908,28,BR,No hotel found
588,Severo-Yeniseyskiy,60.3725,93.0408,86,RU,Актолик
589,Quang Ngai,15.1167,108.8000,82,VN,Ngoc Anh
590,Peterhead,57.5058,-1.7981,84,GB,Albert Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Country", "Hotel Name"],
    geo=True,
    tiles= "OSM",
    frame_width=700,
    frame_height=500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)